In [33]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import joblib
df = pd.read_csv('D:\\Users\\flammmes\\Downloads\\Book1.csv')
df['d']=df['d'].str.replace(',','')
df['Rsob']=df['Rsob'].str.replace(',','')
dfn = pd.read_csv('D:\\Users\\flammmes\\Downloads\\composition.csv')


dd = df.d.astype('float')
df.d = df.d.astype('float')
df['Rsob']= df['Rsob'].astype('float')
df['Rsob']= np.abs(df['Rsob'])
x = df.iloc[:,1]/df.iloc[:,8]
mu = 1000*(df.iloc[:,3]*x*x+df.iloc[:,4]*x+df.iloc[:,5])/(df.iloc[:,6]*x+1)
df.insert(9,'μob',mu)
df.insert(12,'Tr',dfn.Tres)

df = df[df['μob']<=50] #50,338

df.μob = 1/np.power(df.μob,1/1.5)
my_new = pd.DataFrame({
    'Pb':df.Pb,'μob':df.μob ,'Rs':df.Rsob
})
bb = my_new[my_new['Rs'].isnull().values]
aa = my_new[my_new['Rs'].notnull().values]
dat = aa.iloc[:,0:2].to_numpy()
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
dat =scaler.fit_transform(dat)
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(n_estimators=100)
forest_reg.fit(dat, aa.Rs)

dat2 =bb.iloc[:,0:2].to_numpy()
dat2 =scaler.transform(dat2)
df['Rsob'][df['Rsob'].isnull().values] = forest_reg.predict(dat2)
df.μob = 1/np.power(df.μob,1.5)

df.μob = np.log(df.μob)
my_new = pd.DataFrame({
    'μob':df.μob, 'STO':df.STO
})
bb = my_new[my_new['STO'].isnull().values]
aa = my_new[my_new['STO'].notnull().values]
dat = aa.μob.to_numpy().reshape(-1,1)
scaler2 = StandardScaler()
dat =scaler2.fit_transform(dat)
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(dat, aa.STO)
dat2 = bb.μob.to_numpy().reshape(-1,1)
dat2 =scaler2.transform(dat2)
df['STO'][df['STO'].isnull().values] = lin_reg.predict(dat2)
df.μob = np.exp(df.μob)


my_new = pd.DataFrame({
    'Pb':df.Pb,'Pmax':df.Pmax ,'Rs':df.Rsob,'Tr':df.Tr
})
bb = my_new[my_new['Tr'].isnull().values]
aa = my_new[my_new['Tr'].notnull().values]
dat = aa.iloc[:,0:3].to_numpy()
scaler3 = StandardScaler()
dat =scaler3.fit_transform(dat)
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(n_estimators=100)
forest_reg.fit(dat, aa.Tr)
dat2 =bb.iloc[:,0:3].to_numpy()
dat2 =scaler3.transform(dat2)
df['Tr'][df['Tr'].isnull().values] = forest_reg.predict(dat2)

df["μob_cat"] = pd.cut(df["μob"],
                               bins=[0., 5, np.inf],
                               labels=[1, 2])
from sklearn.model_selection import StratifiedShuffleSplit
c=[]
for i in range(100):
    split = StratifiedShuffleSplit(n_splits=1, test_size=0.2,random_state=i)
    for train_index, test_index in split.split(df, df["μob_cat"]):
        strat_train_set = df.iloc[train_index]
        strat_test_set = df.iloc[test_index]

    for set_ in (strat_train_set, strat_test_set):
        set_.drop("μob_cat", axis=1, inplace=True)
        #set_.drop("μob", axis=1, inplace=True)
    df1 = strat_train_set.copy()
    df2 = strat_test_set.copy()
    df1["μob_cat"] = pd.cut(df1["μob"],
                                   bins=[0., 5, np.inf],
                                   labels=[1, 2])
    split = StratifiedShuffleSplit(n_splits=1, test_size=0.2,random_state=i)
    for ti, tei in split.split(df1, df1["μob_cat"]):
        strat_tr_set = df1.iloc[ti]
        strat_val_set = df1.iloc[tei]
    for set_ in (strat_tr_set, strat_val_set):
        set_.drop("μob_cat", axis=1, inplace=True)
        #set_.drop("μob", axis=1, inplace=True)
    df_train = strat_tr_set.copy()
    df_val = strat_val_set.copy()

    def gen_func(a):

        dflist = []
        for i in range(len(a)):

            
            correlations_select = pd.DataFrame({'P':P_i,'μ':mu_i,'Rsob':instance[10],'API':instance[11],'Tr':instance[12]})
            pressure_bubble_point = correlations_select.iloc[0,0]*np.linspace(1,1,20)
            visc_bubble_point = correlations_select.iloc[0,1]*np.linspace(1,1,20)
            correlations_select.insert(0,"Pb",pressure_bubble_point)
            correlations_select.insert(2,'μob',visc_bubble_point)
            dflist.append(correlations_select)


        totaldf = pd.concat(dflist)
        totaldf.Pb = totaldf.Pb*145.038
        totaldf.P = totaldf.P*145.038
        totaldf.insert(2,'ΔP', totaldf.P-totaldf.Pb)

        totaldf.API = 141.5*1000/totaldf.API -131.5
        totaldf.Rsob = totaldf.Rsob*5.614583333333333
        #totaldf = totaldf[totaldf.μ != totaldf.μob]
        #totaldf.drop('P',axis=1,inplace=True)
        return totaldf
    test_data = gen_func(df2)
    train_data = gen_func(df_train)
    val_data = gen_func(df_val)

    #y = test_data[test_data.ΔP==0]
    #x =  val_data[val_data.ΔP==0]
    #train_data = pd.concat([train_data,y,x])
    all_data = gen_func(df)
    for a in [test_data,train_data,val_data,all_data]:
        a.μ = np.sqrt(np.log(a.μ/a.μob+0.0012))
        a.ΔP = np.power(a.ΔP,0.5)
        #a.ΔP = np.log(a.ΔP+1)
        #a.insert(0,'x1',a.ΔP*np.power(a.P/a.Pb,0.2))
        a.μob = a.ΔP*np.power(a.μob,0.07)

        #a.insert(4,'logμob',a.μob*np.log(a.μob))
        a.Pb = a.ΔP/np.log(a.Pb+400)
        a.P = a.ΔP*np.power(a.P,-0.15)
        a.API = a.ΔP/np.power(a.API,0.3)
        a.Rsob = a.ΔP/np.log(a.Rsob+150)
        #a.drop('μob',axis=1,inplace=True)
        a.Tr = a.ΔP*np.power(a.Tr,-1.)
    from sklearn.preprocessing import StandardScaler
    from sklearn.preprocessing import PolynomialFeatures

    X_train = train_data.drop(["μ"], axis=1) 
    y_train = train_data["μ"].copy()
    X_val = val_data.drop(["μ"], axis=1) 
    y_val = val_data["μ"].copy()
    X_test = test_data.drop(["μ"], axis=1) # drop labels for training set
    y_test = test_data["μ"].copy()

    poly_features = PolynomialFeatures(degree=2, include_bias=False)
    X_train_poly = poly_features.fit_transform(X_train)
    X_val_poly = poly_features.fit_transform(X_val)
    X_test_poly = poly_features.fit_transform(X_test)
    poly_scaler = StandardScaler()
    X_train_poly = poly_scaler.fit_transform(X_train_poly)
    X_val_poly = poly_scaler.transform(X_val_poly)
    X_test_poly = poly_scaler.transform(X_test_poly)

    scaler = StandardScaler()
    X_train= scaler.fit_transform(X_train)
    X_val= scaler.transform(X_val)
    X_test= scaler.transform(X_test)
    X_features_full = [X_train,X_val,X_test]
    X_features_poly_full = [X_train_poly,X_val_poly,X_test_poly]
    y_labels_full = [y_train,y_val,y_test]
    totaldf_full = gen_func(df2)
    all_data_full = gen_func(df)

    from sklearn.metrics import mean_squared_error
    from sklearn.metrics import mean_absolute_percentage_error
    from sklearn.svm import SVR
    X_features = X_features_full
    X_features_poly = X_features_poly_full
    y_labels = y_labels_full
    from sklearn.linear_model import Ridge
    ridge_reg = Ridge(alpha=0.2, solver="cholesky")
    ridge_reg.fit(X_features_poly[0], y_labels[0])
    svm_rbf_reg = SVR(kernel="rbf",C=1,epsilon=0.001)
    svm_rbf_reg.fit(X_features[0], y_labels[0])
    forest_reg = RandomForestRegressor(n_estimators=100)
    forest_reg.fit(X_features[0], y_labels[0])
    from sklearn.ensemble import VotingRegressor
    voting_reg = VotingRegressor(
        estimators=[('fr', forest_reg), ('svm', svm_rbf_reg)])
    voting_reg.fit(X_features[0],y_labels[0])
    import tensorflow as tf
    from tensorflow import keras
    model = keras.models.Sequential([
        keras.layers.Dense(500, activation="relu", input_shape=X_features[0].shape[1:]),
        keras.layers.Dense(1)
    ])
    for j in range(2):
        model.compile(loss="mean_absolute_percentage_error", optimizer=keras.optimizers.Adam())
        history = model.fit(X_features[0], y_labels[0], epochs=20,batch_size=16, validation_data=(X_features[1], y_labels[1]))
    totaldf = totaldf_full
    totaldf.insert(7,'poly ridge μ', totaldf.μob*(np.exp(np.power(ridge_reg.predict(X_features_poly[2]),2)) -0.0012))
    totaldf.insert(8,'rbf μ', totaldf.μob*(np.exp(np.power(svm_rbf_reg.predict(X_features[2]),2)) -0.0012))
    totaldf.insert(9,'forest μ', totaldf.μob*(np.exp(np.power(forest_reg.predict(X_features[2]),2)) -0.0012))
    totaldf.insert(10,'voting μ', totaldf.μob*(np.exp(np.power(voting_reg.predict(X_features[2]),2)) -0.0012))
    totaldf.insert(11,'sequential', (np.exp(np.power(model.predict(X_features[2]),2)) -0.0012))
    totaldf.sequential = totaldf.sequential*totaldf.μob
    totaldf.insert(12,'Almehaideb', totaldf.μob*np.power(totaldf.P/totaldf.Pb,0.134819+1.94345*np.power(10,-4.0)*totaldf.Rsob-1.93106*np.power(10,-9.0)*np.power(totaldf.Rsob,2)))
    X = 0.776644115+0.987658646*np.log10(totaldf.μob)-0.190564677*np.log10(totaldf.Rsob)+0.009147711*totaldf.μob*np.log10(totaldf.Rsob)-0.000019111*(totaldf.P-totaldf.Pb)
    totaldf.insert(13,'Dindoruk and Christman',totaldf.μob +0.00006334*totaldf.ΔP*np.power(10,X))
    def Stats_matrix(a):
        absolute_average_error = []
        for i in range(7):
            absolute_average_error.append(np.average(np.absolute((a.iloc[:,i+7]-a.μ)/a.μ*100)))
        statistics  = pd.Series(list(absolute_average_error))
        return statistics

    Stats_matrix(totaldf)

    y = Stats_matrix(totaldf).values.tolist()
    c.append(y)
    

C:\Users\flammmes\AppData\Local\Temp/ipykernel_8124/668762479.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Rsob'][df['Rsob'].isnull().values] = forest_reg.predict(dat2)
C:\Users\flammmes\AppData\Local\Temp/ipykernel_8124/668762479.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['STO'][df['STO'].isnull().values] = lin_reg.predict(dat2)
C:\Users\flammmes\AppData\Local\Temp/ipykernel_8124/668762479.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#

Epoch 1/20
455/455 [==============================] - 1s 2ms/step - loss: 38.1806 - val_loss: 20.0772
Epoch 2/20
455/455 [==============================] - 1s 1ms/step - loss: 29.2310 - val_loss: 28.5108
Epoch 3/20
455/455 [==============================] - 1s 2ms/step - loss: 26.1671 - val_loss: 22.0176
Epoch 4/20
455/455 [==============================] - 1s 1ms/step - loss: 27.0770 - val_loss: 21.4671
Epoch 5/20
455/455 [==============================] - 1s 2ms/step - loss: 22.6839 - val_loss: 65.8224
Epoch 6/20
455/455 [==============================] - 1s 2ms/step - loss: 22.8251 - val_loss: 17.9157
Epoch 7/20
455/455 [==============================] - 1s 1ms/step - loss: 21.3217 - val_loss: 25.5057
Epoch 8/20
455/455 [==============================] - 1s 2ms/step - loss: 20.1796 - val_loss: 49.6709
Epoch 9/20
455/455 [==============================] - 1s 1ms/step - loss: 19.2458 - val_loss: 16.2531
Epoch 10/20
455/455 [==============================] - 1s 1ms/step - loss: 19.1960

C:\Users\flammmes\AppData\Roaming\Python\Python310\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
C:\Users\flammmes\AppData\Roaming\Python\Python310\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


Epoch 1/20
455/455 [==============================] - 1s 2ms/step - loss: 36.7653 - val_loss: 80.3591
Epoch 2/20
455/455 [==============================] - 1s 2ms/step - loss: 33.0642 - val_loss: 42.3757
Epoch 3/20
455/455 [==============================] - 1s 2ms/step - loss: 30.3715 - val_loss: 20.7993
Epoch 4/20
455/455 [==============================] - 1s 1ms/step - loss: 26.7999 - val_loss: 23.2743
Epoch 5/20
455/455 [==============================] - 1s 2ms/step - loss: 24.5259 - val_loss: 26.0189
Epoch 6/20
455/455 [==============================] - 1s 1ms/step - loss: 22.4326 - val_loss: 27.6053
Epoch 7/20
455/455 [==============================] - 1s 1ms/step - loss: 22.6329 - val_loss: 21.0197
Epoch 8/20
455/455 [==============================] - 1s 2ms/step - loss: 19.4709 - val_loss: 17.4724
Epoch 9/20
455/455 [==============================] - 1s 1ms/step - loss: 20.4118 - val_loss: 16.6964
Epoch 10/20
455/455 [==============================] - 1s 1ms/step - loss: 20.9919

C:\Users\flammmes\AppData\Roaming\Python\Python310\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
C:\Users\flammmes\AppData\Roaming\Python\Python310\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


...

Epoch 1/20
455/455 [==============================] - 2s 3ms/step - loss: 39.0138 - val_loss: 25.6450
Epoch 2/20
455/455 [==============================] - 1s 2ms/step - loss: 27.0965 - val_loss: 88.5936
Epoch 3/20
455/455 [==============================] - 1s 2ms/step - loss: 29.1058 - val_loss: 33.5975
Epoch 4/20
455/455 [==============================] - 1s 2ms/step - loss: 25.2790 - val_loss: 19.6164
Epoch 5/20
455/455 [==============================] - 1s 2ms/step - loss: 25.0832 - val_loss: 22.5560
Epoch 6/20
455/455 [==============================] - 1s 3ms/step - loss: 21.9334 - val_loss: 18.9731
Epoch 7/20
455/455 [==============================] - 1s 3ms/step - loss: 23.0565 - val_loss: 38.2826
Epoch 8/20
455/455 [==============================] - 1s 2ms/step - loss: 21.3115 - val_loss: 14.1795
Epoch 9/20
455/455 [==============================] - 1s 2ms/step - loss: 20.7195 - val_loss: 13.7857
Epoch 10/20
455/455 [==============================] - 1s 2ms/step - loss: 19.9739

C:\Users\flammmes\AppData\Roaming\Python\Python310\site-packages\pandas\core\arraylike.py:364: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\flammmes\AppData\Roaming\Python\Python310\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
C:\Users\flammmes\AppData\Roaming\Python\Python310\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


Epoch 1/20
455/455 [==============================] - 1s 2ms/step - loss: 34.2039 - val_loss: 15.4282
Epoch 2/20
455/455 [==============================] - 1s 2ms/step - loss: 27.9885 - val_loss: 28.8772
Epoch 3/20
455/455 [==============================] - 1s 2ms/step - loss: 25.0650 - val_loss: 24.3827
Epoch 4/20
455/455 [==============================] - 1s 2ms/step - loss: 22.4506 - val_loss: 16.4521
Epoch 5/20
455/455 [==============================] - 1s 2ms/step - loss: 21.0709 - val_loss: 15.5219
Epoch 6/20
455/455 [==============================] - 1s 2ms/step - loss: 19.9276 - val_loss: 21.1699
Epoch 7/20
455/455 [==============================] - 1s 2ms/step - loss: 18.9658 - val_loss: 11.9011
Epoch 8/20
455/455 [==============================] - 1s 2ms/step - loss: 18.3290 - val_loss: 14.2749
Epoch 9/20
455/455 [==============================] - 2s 4ms/step - loss: 18.4041 - val_loss: 12.1600
Epoch 10/20
455/455 [==============================] - 2s 3ms/step - loss: 18.3078

C:\Users\flammmes\AppData\Roaming\Python\Python310\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
C:\Users\flammmes\AppData\Roaming\Python\Python310\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


Epoch 1/20
455/455 [==============================] - 2s 3ms/step - loss: 29.2919 - val_loss: 20.4695
Epoch 2/20
455/455 [==============================] - 1s 2ms/step - loss: 27.4069 - val_loss: 20.7187
Epoch 3/20
455/455 [==============================] - 1s 2ms/step - loss: 22.6923 - val_loss: 14.7822
Epoch 4/20
455/455 [==============================] - 1s 2ms/step - loss: 22.3069 - val_loss: 15.7410
Epoch 5/20
455/455 [==============================] - 1s 2ms/step - loss: 23.5682 - val_loss: 13.5105
Epoch 6/20
455/455 [==============================] - 1s 2ms/step - loss: 21.6670 - val_loss: 18.5413
Epoch 7/20
455/455 [==============================] - 1s 2ms/step - loss: 19.6936 - val_loss: 16.2955
Epoch 8/20
455/455 [==============================] - 1s 2ms/step - loss: 19.8404 - val_loss: 14.4442
Epoch 9/20
455/455 [==============================] - 1s 2ms/step - loss: 18.9909 - val_loss: 16.9002
Epoch 10/20
455/455 [==============================] - 1s 2ms/step - loss: 18.2630

In [38]:
f2 = pd.DataFrame(c)

In [39]:
%store f2

Stored 'f2' (DataFrame)
